In [52]:
import numpy as np
from clifford.g3 import *
from clifford.tools.g3 import generate_rotation_rotor, random_unit_vector
import copy
rng = np.random.default_rng()
#blades # show blades

In [35]:
def pointsToAlgebra(array : np.array):
    out = list()
    for v in array:
        p = v[0]*e1 + v[1]*e2 + v[2]*e3
        out.append(p)
    
    return out
    
def randomG3Multivector(random_vector3 = None):
    if random_vector3 is None:
        random_vector3 = rng.standard_normal(3)
    return random_vector3[0]*e1 + random_vector3[0]*e2 + random_vector3[0]*e3 
        

#### Generate model cloud, ground truth rotor and measured point clouds in 3D

In [36]:
# Model point cloud
np.random.seed(0)
size = 100
scale = 3
model_cloud = np.random.rand(100,3) * scale
model_cloud_g3 = pointsToAlgebra(model_cloud)

# Generate a ground truth motor
theta = np.pi / 4
t = -5*e1 + 3*e2 -np.pi*e3
M = generate_rotation_rotor(theta, e1, e2)

# Measured point cloud: obtained by adding rotating each point in the cloud
# and adding isotropic gaussian noise
measure_cloud_g3 = [ (M*e*~M) + randomG3Multivector()  for e in model_cloud_g3]

#### Define initial values for multirotor 

This is the initial rotor estimate. Also define some iteration parameters like the maximum number of iterations and residual threshold to stop iterations

In [53]:
max_iters = 200 # maximum number of iterations allowed to stop iterating
residual_thresh = float(0.05) # residual threshold to stop iterating
r_init = 0.5 + 0.5*e12 + 0.5*e12 + 0.5*e23 # initial value of the rotor
r =  0.5 + 0.5*e12 + 0.5*e12 + 0.5*e23 # optimized value of the rotor
mu_galms = 8


iter = 0 # current iteration number
residual = float(10.0) # current  residual
while(iter < max_iters and residual > residual_thresh):

    for i, (p , q) in enumerate(zip(model_cloud_g3, measure_cloud_g3)):
        r = r_init + mu_galms * (q ^ (r_init * p * ~r_init))*r_init
        r = r.normal()
        r_init = copy.deepcopy(r)

        # Get the residual
        residual = (r*~r)(0)[0]

    iter += 1

    if (iter > max_iters):
        print(f"Finished due to maximum iterations: {iter}/{max_iters}")
    
    if (residual < residual_thresh):
        print(f"Finished due to residual: {residual}/{residual_thresh}")

print(f"Iters: {iter}, Residual: {residual}")

/home/juanmedrano_eng/repos/clifford/.env/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Treating MultiVector objects like a sequence is deprecated. To access the coefficients as a sequence, use the `.value` attribute.


KeyboardInterrupt: 

In [46]:
M

0.92388 - (0.38268^e12)